In [5]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("NYPD_Complaint_Data_Current_YTD.csv")
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,579468609,12/31/2017,23:55:00,NaN,NaN,12/31/2017,118,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,...,67,NaN,STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,822699000,12/31/2017,23:30:00,NaN,NaN,12/31/2017,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,...,47,FRONT OF,RESIDENCE - APT. HOUSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187081621,12/31/2017,23:10:00,12/31/2017,23:15:00,12/31/2017,578,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",...,23,INSIDE,RESIDENCE - PUBLIC HOUSING,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,967006956,12/31/2017,23:05:00,12/31/2017,23:15:00,12/31/2017,118,DANGEROUS WEAPONS,793.0,WEAPONS POSSESSION 3,...,73,INSIDE,RESIDENCE-HOUSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,524297993,12/31/2017,23:00:00,NaN,NaN,12/31/2017,118,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,...,61,FRONT OF,RESIDENCE - APT. HOUSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
dfh=pd.read_csv("NYPD_Complaint_Data_Historic.csv")

/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
dft1=pd.concat([df,dfh])

In [6]:
#I don't need greater level of granularity location than a borough so I drop cooridates columns
# I don't need information about park name or jurisdication responsible for the incident so I drop those columns as well
#dft1 = dft.drop(columns = ['X_COORD_CD','Y_COORD_CD','Latitude','Longitude',"Lat_Lon", 'PARKS_NM','JURIS_DESC'])
#keeping the coordinates



In [7]:
#hadevelopt has many NaNs so I drop this column too 
dft1=dft1.drop(columns="HADEVELOPT")

In [ ]:
#the information about ending date can be used to fill out missing information about CMPLNT_FR_DT
df3=dft1[['CMPLNT_FR_DT','CMPLNT_TO_DT','RPT_DT']]
#df3[df3['CMPLNT_FR_DT'].isnull()]
df3.tail()

In [12]:
#filling out nans in complaint date
def datef(row):
    
    if pd.isnull(row["CMPLNT_FR_DT"]):
        if pd.notnull((row["CMPLNT_TO_DT"])):
            return row["CMPLNT_TO_DT"]
        elif pd.notnull(row['RPT_DT']):
            return row['RPT_DT']
        else: 
            return row["CMPLNT_FR_DT"]
    else: 
        return row["CMPLNT_FR_DT"]
        

In [13]:
dft1["CMPLNT_FR_DT"]=dft1.apply(datef,axis=1)

In [ ]:
dft1.count()


There is two more columns I will get rid of CMPLNT_TO_DT and CMPLNT_TO_TM - I only need a date of crime and optionally a time. 
I can also drop RPT_DT since I am more interested in the date of the crime and usually reporting date is close to date of crime. 
LOC_OF_OCCUR_DESC is also incomplete in one third so I will drop this column too. 


In [14]:
dft2=dft1.drop(columns=['CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT','LOC_OF_OCCUR_DESC'])
#dft2.info()

In [16]:
dft2.rename(columns={'CMPLNT_NUM':'ComplaintID', 'CMPLNT_FR_DT':'Date','CMPLNT_FR_TM':'Time',"KY_CD":"Offence Code",'OFNS_DESC':'Description','PD_CD':'Internal Code',
 'PD_DESC':'Internal Description',
 'CRM_ATPT_CPTD_CD':'Status',
 'LAW_CAT_CD':'Offence Level',
 'BORO_NM':'Borough',
 'ADDR_PCT_CD':'Neighborhood',
 'PREM_TYP_DESC':'Premise Description'}, inplace=True)

dft2.reset_index(inplace=True)
dft2.drop(columns='index')


,ComplaintID,Date,Time,Offence Code,Description,Internal Code,Internal Description,Status,Offence Level,JURIS_DESC,Borough,Neighborhood,Premise Description,PARKS_NM,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,579468609,12/31/2017,23:55:00,118,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,COMPLETED,FELONY,N.Y. POLICE DEPT,BROOKLYN,67.0,STREET,NaN,NaN,NaN,NaN,NaN,NaN
1,822699000,12/31/2017,23:30:00,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,BRONX,47.0,RESIDENCE - APT. HOUSE,NaN,NaN,NaN,NaN,NaN,NaN
2,187081621,12/31/2017,23:10:00,578,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",COMPLETED,VIOLATION,N.Y. HOUSING POLICE,MANHATTAN,23.0,RESIDENCE - PUBLIC HOUSING,NaN,NaN,NaN,NaN,NaN,NaN
3,967006956,12/31/2017,23:05:00,118,DANGEROUS WEAPONS,793.0,WEAPONS POSSESSION 3,COMPLETED,FELONY,N.Y. POLICE DEPT,BROOKLYN,73.0,RESIDENCE-HOUSE,NaN,NaN,NaN,NaN,NaN,NaN
4,524297993,12/31/2017,23:00:00,118,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,COMPLETED,FELONY,N.Y. POLICE DEPT,BROOKLYN,61.0,RESIDENCE - APT. HOUSE,NaN,NaN,NaN,NaN,NaN,NaN
5,656720558,12/31/2017,23:00:00,351,CRIMINAL MISCHIEF & RELATED OF,254.0,"MISCHIEF, CRIMINAL 4, OF MOTOR",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,BROOKLYN,61.0,STREET,NaN,NaN,NaN,NaN,NaN,NaN
6,259728915,12/31/2017,23:00:00,118,DANGEROUS WEAPONS,793.0,WEAPONS POSSESSION 3,COMPLETED,FELONY,N.Y. HOUSING POLICE,BROOKLYN,83.0,RESIDENCE - PUBLIC HOUSING,NaN,NaN,NaN,NaN,NaN,NaN
7,643915940,12/31/2017,23:00:00,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,QUEENS,109.0,RESIDENCE-HOUSE,NaN,NaN,NaN,NaN,NaN,NaN
8,309983236,12/31/2017,22:53:00,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,STATEN ISLAND,120.0,STREET,NaN,NaN,NaN,NaN,NaN,NaN
9,322611002,12/31/2017,22:50:00,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,N.Y. HOUSING POLICE,QUEENS,101.0,RESIDENCE - PUBLIC HOUSING,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# to save some memory I will assign Borough, Status, Offence Level as 'category'
dft2['Borough']=dft2['Borough'].astype('category')
dft2['Offence Code']=dft2['Offence Code'].astype('category')
dft2['Internal Code']=dft2['Internal Code'].astype('category')
dft2['Status']=dft2['Status'].astype('category')
dft2['Offence Level']=dft2['Offence Level'].astype('category')
dft2['Neighborhood']=dft2['Neighborhood'].astype('category')

In [ ]:
dft2.drop(columns=['Unnamed: 0','index'],inplace=True)

In [20]:
dft2['Year']=dft2.Date.str[-4:]
dft2['Month']=dft2.Date.str[-7:-5]


In [ ]:
dft2[dft2['Year']=='1015']

In [ ]:
dft2.groupby(by='Year')['ComplaintID'].count()

There is some weird year dates that probably were typed by mistake, also I am not interested in data before 2006

In [21]:
dft2=dft2[(dft2['Year']=='2017') | (dft2['Year']=='2016' ) | (dft2['Year']=='2015' )|(dft2['Year']=='2014')|(dft2['Year']=='2013' )|(dft2['Year']=='2012' )|(dft2['Year']=='2011' )|(dft2['Year']=='2010' )|(dft2['Year']=='2009' )|(dft2['Year']=='2008' )|(dft2['Year']=='2007' )|(dft2['Year']=='2006' )]

In [23]:
dft2['Datetime'] = dft2['Date'] + ' ' + dft2['Time']

/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
dft2.tail()

,index,ComplaintID,Date,Time,Offence Code,Description,Internal Code,Internal Description,Status,Offence Level,...,Premise Description,PARKS_NM,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,Year,Month,Datetime
6048790,5580029,593729461,01/01/2015,04:30:00,105,ROBBERY,382.0,"ROBBERY,NECKCHAIN/JEWELRY",COMPLETED,FELONY,...,STREET,NaN,1032890.0,216576.0,40.761002,-73.824423,"(40.761001984, -73.824422813)",2015,01,01/01/2015 04:30:00
6048791,5580030,582350583,01/01/2015,03:50:00,105,ROBBERY,399.0,"ROBBERY,COMMERCIAL UNCLASSIFIED",COMPLETED,FELONY,...,HOTEL/MOTEL,NaN,998372.0,240146.0,40.825818,-73.948975,"(40.825817778, -73.948974825)",2015,01,01/01/2015 03:50:00
6048792,5580031,258046495,01/01/2015,01:25:00,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,...,TRANSIT - NYC SUBWAY,NaN,1014468.0,238156.0,40.820315,-73.890825,"(40.820315396, -73.890824603)",2015,01,01/01/2015 01:25:00
6048793,5580032,640212578,01/01/2015,00:30:00,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,...,BAR/NIGHT CLUB,NaN,1030529.0,214093.0,40.754199,-73.832963,"(40.754199468, -73.832962523)",2015,01,01/01/2015 00:30:00
6048794,5580033,496500431,06/30/2014,12:00:00,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,COMPLETED,MISDEMEANOR,...,RESIDENCE - APT. HOUSE,NaN,1009735.0,261272.0,40.883777,-73.907837,"(40.883776851, -73.907836928)",2014,30,06/30/2014 12:00:00


In [25]:
dft2["Date"] =pd.to_datetime(dft2['Date'], format='%m/%d/%Y')

/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
dft2.set_index('Date',inplace=True)

In [28]:
dft2.drop(columns='Datetime',inplace=True)
dft2['Borough']=dft2['Borough'].astype('category')
dft2['Offence Code']=dft2['Offence Code'].astype('category')
dft2['Internal Code']=dft2['Internal Code'].astype('category')
dft2['Status']=dft2['Status'].astype('category')
dft2['Offence Level']=dft2['Offence Level'].astype('category')
dft2['Neighborhood']=dft2['Neighborhood'].astype('category')
dft2['Year']=dft2['Year'].astype('category')
dft2['Month']=dft2['Month'].astype('category')

/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [30]:
#changing numerical data into str and then into categorical data
#dft2['Neighborhood']=dft2["Neighborhood"].astype(int).astype(str).astype('category')
dft2['Internal Code']=dft2["Internal Code"].astype(int).astype(str).astype('category')
dft2['Offence Code']=dft2["Offence Code"].astype(str).astype('category')

/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/michalczapski/anaconda/envs/aind/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
#saving to pickle
dft2.to_pickle("NYPD_data_clean_coord")

In [ ]:
#saving to CSV
dft2.to_csv("NYPD_data_clean_coord.csv")